In [1]:

import sys; 
CPBdir = "../../../../CPB/CPlantBox"
sys.path.append(CPBdir+"/src");
sys.path.append(CPBdir);
sys.path.append("../../..");sys.path.append(".."); 
sys.path.append(CPBdir+"/src/python_modules");
sys.path.append("../build-cmake/cpp/python_binding/") # dumux python binding
sys.path.append("../../build-cmake/cpp/python_binding/")
sys.path.append("../modules/") # python wrappers 
from helpuqrMaster import *

In [2]:

def setKrKx_Xylem(TairC, RH,r): #inC
    #mg/cm3
    hPa2cm = 1.0197
    dEauPure = (999.83952 + TairC * (16.952577 + TairC * 
        (- 0.0079905127 + TairC * (- 0.000046241757 + TairC * 
        (0.00000010584601 + TairC * (- 0.00000000028103006)))))) /  (1 + 0.016887236 * TairC)
    siPhi = (30 - TairC) / (91 + TairC)
    siEnne=0
    mu =  pow(10, (- 0.114 + (siPhi * (1.1 + 43.1 * pow(siEnne, 1.25) )))) 
    mu = mu /(24*60*60)/100/1000; #//mPa s to hPa d, 1.11837e-10 hPa d for pure water at 293.15K
    mu = mu * hPa2cm #hPa d to cmh2o d 

    #number of vascular bundles
    VascBundle_leaf = 32
    VascBundle_stem = 52
    VascBundle_root = 1 #valid for all root type
            
    #radius of xylem type^4 * number per bundle
    rad_x_l_1   = (0.0015 **4) * 2; rad_x_l_2   = (0.0005 **4) * 2   
    rad_x_s_1   = (0.0017 **4) * 3; rad_x_s_2   = (0.0008 **4) * 1     
    rad_x_r0_1  = (0.0015 **4) * 4    
    rad_x_r12_1 = (0.00041**4) * 4; rad_x_r12_2 = (0.00087**4) * 1
    rad_x_r3_1  = (0.00068**4) * 1      

    # axial conductivity [cm^3/day]  
    betaXylX = 1    
    kz_l  = VascBundle_leaf *(rad_x_l_1 + rad_x_l_2)    *np.pi /(mu * 8)  * betaXylX
    kz_s  = VascBundle_stem *(rad_x_s_1 + rad_x_s_2)    *np.pi /(mu * 8) * betaXylX 
    kz_r0 = VascBundle_root * rad_x_r0_1                *np.pi /(mu * 8) * betaXylX  
    kz_r1 = VascBundle_root *(rad_x_r12_1 + rad_x_r12_2)*np.pi /(mu * 8)  * betaXylX
    kz_r2 = VascBundle_root *(rad_x_r12_1 + rad_x_r12_2)*np.pi /(mu * 8)  * betaXylX 
    kz_r3 = VascBundle_root * rad_x_r3_1                *np.pi /(mu * 8) * betaXylX

    #radial conductivity [1/day],0.00014 #
    betaXyl = 1#0.1#0.1
    kr_l  = 3.83e-06 * hPa2cm * betaXyl# init: 3.83e-4 cm/d/hPa
    print("kr_l ",kr_l)
    kr_s  = 0.#1.e-20  * hPa2cm # set to almost 0
    kr_r0 =6.37e-5 * hPa2cm * betaXyl
    kr_r1 =7.9e-5  * hPa2cm * betaXyl
    kr_r2 =7.9e-5  * hPa2cm * betaXyl
    kr_r3 =6.8e-5  * hPa2cm * betaXyl
    l_kr = 0.8 #cm
    r.setKr([[kr_r0,kr_r1,kr_r2,kr_r0],[kr_s,kr_s ],[kr_l]], kr_length_=l_kr) 
    r.setKx([[kz_r0,kz_r1,kz_r2,kz_r0],[kz_s,kz_s ],[kz_l]])
    
    
    Rgaz=8.314 #J K-1 mol-1 = cm^3*MPa/K/mol
    rho_h2o = dEauPure/1000#g/cm3
    Mh2o = 18.05 #g/mol
    MPa2hPa = 10000
    hPa2cm = 1/0.9806806
    #log(-) * (cm^3*MPa/K/mol) * (K) *(g/cm3)/ (g/mol) * (hPa/MPa) * (cm/hPa) =  cm                      
    #p_a = np.log(RH) * Rgaz * rho_h2o * (TairC + 273.15)/Mh2o * MPa2hPa * hPa2cm
    #done withint solve photosynthesis
    #r.psi_air = p_a #*MPa2hPa #used only with xylem
    return r

In [3]:
condition = "dry"
def weather(simDuration, hp):
    vgSoil = [0.059, 0.45, 0.00644, 1.503, 1]
    loam = [0.08, 0.43, 0.04, 1.6, 50]
    Qnigh = 0; Qday = 960e-6 #458*2.1
    if (condition == "wet"):
        Tnigh = 15.8; Tday = 22
        #Tnigh = 13; Tday = 20.7
        #specificHumidity = 0.0097
        RHday = 0.60; RHnigh = 0.88
        Pair = 1010.00 #hPa
        thetaInit = 30/100
        cs = 350e-6
    elif condition == "dry":
        Tnigh = 20.7; Tday = 30.27
        #Tnigh = 15.34; Tday = 23.31
        #specificHumidity = 0.0097# 0.0111
        RHday = 0.7; RHnigh =0.7#0.44; RHnigh = 0.78
        Pair = 1070.00 #hPa
        thetaInit = 20/100   
        cs = 350e-6
    else:
        print("condition",condition)
        raise Exception("condition not recognised")
    coefhours = sinusoidal(simDuration)
    RH_ = RHnigh + (RHday - RHnigh) * coefhours
    TairC_ = Tnigh + (Tday - Tnigh) * coefhours
    Q_ = Qnigh + (Qday - Qnigh) * coefhours
     #co2 paartial pressure at leaf surface (mol mol-1)
    #390, 1231
    #RH = 0.5 # relative humidity
    es =  6.112 * np.exp((17.67 * TairC_)/(TairC_ + 243.5))
    ea = es*RH_#qair2ea(specificHumidity,  Pair)
    assert ea < es
    #RH = ea/es
    assert ((RH_ > 0) and(RH_ < 1))
    bl_thickness = 1/1000 #m
    diffusivity= 2.5e-5#m2/sfor 25°C
    rbl =bl_thickness/diffusivity #s/m 13
    #cs = 350e-6
    Kcanopymean = 1e-1 # m2/s
    meanCanopyL = (2/3) * hp /2
    rcanopy = meanCanopyL/Kcanopymean
    windSpeed = 2 #m/s
    zmzh = 2 #m
    karman = 0.41 #[-]

    rair = 1
    if hp > 0:
        rair = np.log((zmzh - (2/3)*hp)/(0.123*hp)) * np.log((zmzh - (2/3)*hp)/(0.1*hp)) / (karman*karman*windSpeed)
        #print()
        #raise Exception


    pmean = theta2H(vgSoil, thetaInit)

    weatherVar = {'TairC' : TairC_,'TairK' : TairC_ + 273.15,'Pair':Pair,"es":es,
                    'Qlight': Q_,'rbl':rbl,'rcanopy':rcanopy,'rair':rair,"ea":ea,
                    'cs':cs, 'RH':RH_, 'p_mean':pmean, 'vg':loam}
    print("Env variables at", round(simDuration//1),"d",round((simDuration%1)*24),"hrs :\n", weatherVar)
    return weatherVar

In [4]:
def resistance2conductance(resistance,r):
    resistance = resistance* (1/100) #[s/m] * [m/cm] = [s/cm]
    resistance = resistance * r.R_ph * weatherX["TairK"] / r.Patm # [s/cm] * [K] * [hPa cm3 K−1 mmol−1] * [hPa] = [s] * [cm2 mmol−1]
    resistance = resistance * (1000) * (1/10000)# [s cm2 mmol−1] * [mmol/mol] * [m2/cm2] = [s m2 mol−1]
    return 1/resistance

In [5]:
def initPlant(simDuration):
    weatherInit = weather(0,0)
    #simDuration = 25 # [day] init simtime
    #spellDuration = 5
    simMax = 26#simStartSim+ spellDuration
    depth = 40
    dt = 1/24 #10min
    verbose = True

    # plant system 
    pl = pb.MappedPlant(seednum = 2) #pb.MappedRootSystem() #pb.MappedPlant()
    #pl2 = pb.MappedPlant(seednum = 2) #pb.MappedRootSystem() #pb.MappedPlant()
    path = CPBdir+"/modelparameter/plant/"
    name = "Triticum_aestivum_test_2021"#"Triticum_aestivum_adapted_2021"#

    pl.readParameters(path + name + ".xml")
    #pl2.readParameters(path + name + ".xml")



    #raise Exception
    sdf = pb.SDF_PlantBox(np.Inf, np.Inf, depth )

    pl.setGeometry(sdf) # creates soil space to stop roots from growing out of the soil
    #pl2.setGeometry(sdf) # creates soil space to stop roots from growing out of the soil


    pl.initialize(verbose = True)#, stochastic = False)
    pl.simulate(simDuration, False)#, "outputpm15.txt")

    #raise Exception
    """ Coupling to soil """



    min_b = [-3./2, -12./2, -41.]#distance between wheat plants
    max_b = [3./2, 12./2, 0.]
    rez = 0.5
    cell_number = [int(6*rez), int(24*rez), int(40*rez)]#1cm3? 
    layers = depth; soilvolume = (depth / layers) * 3 * 12
    k_soil = []
    initial = weatherInit["p_mean"]#mean matric potential [cm] pressure head

    p_mean = initial
    p_bot = p_mean + depth/2
    p_top = initial - depth/2
    sx = np.linspace(p_top, p_bot, depth)
    picker = lambda x,y,z : max(int(np.floor(-z)),-1) 
    sx_static_bu = sx    
    pl.setSoilGrid(picker)  # maps segment



    """ Parameters phloem and photosynthesis """
    r = PhloemFluxPython(pl,psiXylInit = min(sx),ciInit = weatherInit["cs"]*0.5) #XylemFluxPython(pl)#
    #r2 = PhloemFluxPython(#pl2,psiXylInit = min(sx),ciInit = weatherInit["cs"]*0.5) #XylemFluxPython(pl)#

    r = setKrKx_phloem(r)

    r.oldciEq = True

    r.Rd_ref = 0 #to avoid error (C < 0 in meso, mention this in paper)
    r.g0 = 8e-3
    r.VcmaxrefChl1 =1.28#/2
    r.VcmaxrefChl2 = 8.33#/2
    r.a1 = 0.6/0.4#0.7/0.3#0.6/0.4 #ci/(cs - ci) for ci = 0.6*cs
    r.a3 = 1.5
    r.alpha = 0.4#0.2#/2
    r.theta = 0.6#0.9#/2
    r.k_meso = 1e-3#1e-4
    r.setKrm2([[2e-5]])
    r.setKrm1([[10e-2]])#([[2.5e-2]])
    r.setRhoSucrose([[0.51],[0.65],[0.56]])
    rootFact = 2
    r.setRmax_st([[2.4*rootFact,1.5*rootFact,0.6*rootFact,2.4*rootFact],[2.,2.],[8.]])#6.0#*6 for roots, *1 for stem, *24/14*1.5 for leaves
    #r.setRmax_st([[12,9.0,6.0,12],[5.,5.],[15.]])
    r.KMrm = 0.1
    r.sameVolume_meso_st = False
    r.sameVolume_meso_seg = True
    r.withInitVal =True
    r.initValST = 0.#0.6#0.0
    r.initValMeso = 0.#0.9#0.0
    r.beta_loading = 0.6
    r.Vmaxloading = 0.05 #mmol/d, needed mean loading rate:  0.3788921068507634
    r.Mloading = 0.2
    r.Gr_Y = 0.8
    r.CSTimin = 0.4
    r.surfMeso=0.0025
    r.leafGrowthZone = 2 # cm
    r.StemGrowthPerPhytomer = True # 
    r.psi_osmo_proto = -10000*1.0197 #schopfer2006
    r.fwr = 0

    r.cs = weatherInit["cs"]

    #r.r_forPhloem(24/14*1.5, 4)
    #r.r_forPhloem(24/14, 3)
    #r.r_forPhloem(6, 2) #because roots will have high C_ST less often
    r.expression = 6
    r.update_viscosity = True
    r.solver = 1
    r.atol = 1e-10
    r.rtol = 1e-6
    #r.doNewtonRaphson = False;r.doOldEq = False
    SPAD= 41.0
    chl_ = (0.114 *(SPAD**2)+ 7.39 *SPAD+ 10.6)/10
    r.Chl = np.array( [chl_]) 
    r.Csoil = 1e-4
    return r, sx


In [6]:
simDuration = 15
initr = initPlant(simDuration)

Env variables at 0 d 0 hrs :
 {'TairC': 25.485, 'TairK': 298.635, 'Pair': 1070.0, 'es': 32.6028355425364, 'Qlight': 0.00048, 'rbl': 40.0, 'rcanopy': 0.0, 'rair': 1, 'ea': 22.821984879775478, 'cs': 0.00035, 'RH': 0.7, 'p_mean': -1142.0710553436932, 'vg': [0.08, 0.43, 0.04, 1.6, 50]}
MappedPlant::initialize 
Seed::initialize: Plant 
Seed::initialize: Basal root type #4 was not defined, using tap root parameters instead
Tiller stem type #4 was not defined, using main stem parameters instead, Exchange zone in roots: kr_st > 0 until 0.8cm from root tip 1 1
Exchange zone in roots: kr > 0 until 0.8cm from root tip


In [7]:
r = initr[0]
sx = initr[1]

In [8]:
hp = max([tempnode[2] for tempnode in r.get_nodes()]) /100 

weatherX = weather(simDuration, hp)
#r.Patm = weatherX["Pair"]
##resistances
#r.g_bl = resistance2conductance(weatherX["rbl"],r) / r.a2_bl
#r.g_canopy = resistance2conductance(weatherX["rcanopy"],r) / r.a2_canopy
#r.g_air = resistance2conductance(weatherX["rair"],r) / r.a2_air

r.Qlight = weatherX["Qlight"] #; TairC = weatherX["TairC"] ; text = "night"


r = setKrKx_Xylem(weatherX["TairC"], weatherX["RH"],r)
r.es = weatherX["es"]

Env variables at 15 d 0 hrs :
 {'TairC': 25.48499999999995, 'TairK': 298.63499999999993, 'Pair': 1070.0, 'es': 32.602835542536305, 'Qlight': 0.00047999999999999486, 'rbl': 40.0, 'rcanopy': 0.9528334523440218, 'rair': 48.62074127571149, 'ea': 22.82198487977541, 'cs': 0.00035, 'RH': 0.7, 'p_mean': -1142.0710553436932, 'vg': [0.08, 0.43, 0.04, 1.6, 50]}
kr_l  3.905451e-06
Kr_st is constant per subtype of organ type, for root, subtype 1 = 0.05 1 day-1 
Kx_st is constant per subtype of organ type, for root, subtype 1 = 1.32591e-13 cm3 day-1 
Across_st is constant per subtype of organ type, for root, subtype 1 = 9.62113e-06 cm2 
krm2 is constant 2e-05 -
krm1 is constant 0.1 -
rhoSucrose is constant per organ type, organ type 2 (root) = 0.51 mmol cm-3
Rmax_st is constant per subtype of organ type, for root, subtype 1 = 3 cm day-1 
Exchange zone in roots: kr > 0 until 0.8cm from root tip


In [20]:
sx

array([-118.57038869, -117.54474767, -116.51910664, -115.49346562,
       -114.46782459, -113.44218356, -112.41654254, -111.39090151,
       -110.36526049, -109.33961946, -108.31397844, -107.28833741,
       -106.26269638, -105.23705536, -104.21141433, -103.18577331,
       -102.16013228, -101.13449126, -100.10885023,  -99.0832092 ,
        -98.05756818,  -97.03192715,  -96.00628613,  -94.9806451 ,
        -93.95500408,  -92.92936305,  -91.90372203,  -90.878081  ,
        -89.85243997,  -88.82679895,  -87.80115792,  -86.7755169 ,
        -85.74987587,  -84.72423485,  -83.69859382,  -82.67295279,
        -81.64731177,  -80.62167074,  -79.59602972,  -78.57038869])

In [19]:
weatherX

{'TairC': 18.900000000000006,
 'TairK': 292.04999999999995,
 'Pair': 1010.0,
 'es': 21.823315124831193,
 'Qlight': 0.000480000000000001,
 'rbl': 40.0,
 'rcanopy': 1.061506687608708,
 'rair': 45.77437033740226,
 'ea': 16.149253192375078,
 'cs': 0.00035,
 'RH': 0.7399999999999998,
 'p_mean': -98.57038869202552,
 'vg': [0.08, 0.43, 0.04, 1.6, 50]}

In [39]:
r.gm = 0.05

In [53]:
r.solve_photosynthesis(sim_time_ = simDuration, sxx_=sx, cells_ = True,RH_ = 0.9,
                verbose_ = False, doLog_ = True,TairC_= weatherX["TairC"] )
#(double sim_time_,std::vector<double> sxx_, bool cells_ , 
	 #std::vector<double> soil_k_, bool doLog_ , int verbose_ , double RH_, double TairC_)

In [54]:
print(min(r.psiXyl),max(r.psiXyl))

-2484.7900697866307 -2361.6949842584386


In [55]:
r.PVD

[0.0,
 0.0,
 0.0,
 0.0,
 3.0853870239177255,
 3.0853194390175247,
 3.0852546861372474,
 3.085192804793536,
 3.0851338272862705,
 3.085077778754389,
 3.085024698691882,
 3.084974616862212,
 3.084927552134012,
 3.0848835323435466,
 3.0848425927800776,
 3.0848047674006054,
 0.0,
 0.0,
 0.0,
 0.0,
 3.0853956853232347,
 3.0853477321486906,
 3.085302546848851,
 3.0852603053236507,
 0.0,
 0.0,
 0.0,
 0.0,
 3.0854310340537836,
 3.0853732759509356,
 3.0853183876163186,
 3.085266400583066,
 3.0852173485729963,
 3.0851712628616994,
 3.0851281743063197,
 3.0850881133211274]

In [57]:
fluxes = np.array(r.outputFlux)
fluxesSoil = r.soilFluxes(simDuration, r.psiXyl, sx, approx=False)

In [58]:
print(sum(fluxesSoil.values()))

-1.7242714607054053


In [59]:
print("Ev",sum(r.Ev))#in cm3 d-1
print("An",np.mean(r.An)*1e6)#An.append
print("Vc",np.mean(r.Vc)*1e6)#Vc.append
print("Vj",np.mean(r.Vj)*1e6)#Vj.append
print("gco2",np.mean(r.gco2))#gco2.append
print("cics",np.mean(r.ci)/r.cs)#cics.append
print("fw",np.mean(r.fw))#fw.append

Ev 1.7242714588250088
An 13.529392012263678
Vc 13.846276323718632
Vj 17.416703207769103
gco2 0.11180207883864542
cics 0.4189520562303277
fw 0.61496331491345


In [34]:
fluxesSoil

{0: 1.0483958178392796e-06,
 1: 8.788048260421024e-07,
 2: 1.2285166154962788e-05,
 3: 1.6532880046957922e-05,
 4: 1.0331024675751964e-05,
 5: 1.1217111013398359e-05,
 6: 9.756526955521605e-06,
 7: 7.490784700139881e-06,
 8: -2.7902217212794543e-06,
 9: -2.7790492512644937e-06,
 10: 1.2775902232972985e-06,
 11: -1.9314088432035812e-06,
 12: -5.040815295408892e-06,
 13: -6.815800937895184e-06,
 14: -8.56379004206603e-06,
 15: -5.0176292262096686e-06,
 16: -1.47879640801498e-06,
 17: -8.078889949384104e-06,
 18: -2.4349896995607693e-06,
 19: -2.1105753934025644e-06,
 20: -1.1675962900766799e-05,
 21: -2.3804209186606975e-06,
 22: -4.0950524375373096e-06,
 23: -1.0992333991388494e-06,
 24: 0.0,
 25: -1.0251768179170097e-06,
 26: 0.0,
 27: 0.0,
 28: -1.1074143516515187e-06,
 29: 0.0,
 30: -3.2162810750168535e-07,
 31: -8.594313012288592e-07,
 32: -1.2119973849001896e-06}

In [37]:
sum(fluxes)

2.6918570294390463e-14